In [37]:
import time
import copy
import random
import sys

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from tqdm import tqdm

%matplotlib inline

In [38]:
epsilon = .5
alpha = 0.05
gamma = 1

ACTIONS = [0,1,2]

num_inv_steps = 3

num_prob_steps = 10

Q_dims = (num_inv_steps, num_prob_steps, len(actions))

fill_probs = np.linspace(0,1,10)

c = 0

max_iter = float('inf')

actions = {'buy': 2, 'sell': 0, 'hold': 1} 
positions = {'flat': 0, 'long': 2, 'short': 1}

In [39]:
fill_probs

array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])

In [40]:
class DataFeed():
    def __init__(self,data_RA):
        self.data_RA = data_RA
        self.rewind()
    def next(self):
        try:
            return self.__gen.__next__()
        except StopIteration as e:
            raise e
    def rewind(self):
        self.__gen = (row for row in self.data_RA)

        csv_path

In [41]:
%ls ..

README.md  RL_dixon/  coursera/  jansen/    workspace/


In [42]:
csv_path = './data/AMZN-L1.csv'
data_RA = np.genfromtxt(csv_path, delimiter =',', dtype=float)
data_generator = DataFeed(data_RA)

In [43]:
def get_initial_state(data_generator):
    data_generator.rewind()
    
    position = positions['flat']
    entry_price = None
    
    ask, ask_depth, bid, bid_depth = data_generator.next()
    
    price = {'bid': bid/1000.0, 'ask': ask/1000.0}
    
    q = bid_depth / (bid_depth + ask_depth)
    q_ind = int(q * num_prob_steps)
    
    initial_state = position, q_ind, price, entry_price
    
    return initial_state

In [44]:
START = get_initial_state(data_generator)
print(START)

(0, 5, {'bid': 2231.8, 'ask': 2239.5}, None)


In [45]:
def step(state, action):
    position, q, price, entry_price = state
    reward = 0
    instant_pnl = 0
    done = False
    
    if fill_probs[q] < np.random.rand():
        fill_bid = True
        fill_ask = False
    else:
        fill_bid = False
        fill_ask = True
    
    if (action == actions['buy']) and (fill_bid):
        reward = -c
        if position == positions['flat']:
            position = positions['long']
            entry_price = price['bid']
        elif position == positions['short']:
            position = positions['flat']
            exit_price = price['bid']
            instant_pnl = entry_price - exit_price
            entry_price = None
        elif position == positions['long']:
            raise ValueError("can't buy, already have")
    
    elif (action == actions['sell'] and fill_ask):
        reward = -c
        
        if position == positions['flat']:
            position = positions['short']
            entry_price = price['ask']
        elif position == positions['long']:
            exit_price = price['ask']
            position = positions['flat']
            instant_pnl = exit_price - entry_price
            entry_price = None
        elif position == positions['short']:
            raise ValueError("Can't sell, already short")
    reward += instant_pnl
    
    try:
        ask, ask_depth, bid, bid_depth = data_generator.next()
        
        price = {'bid': bid/1000.0, 'ask':ask/1000.0}
        
        q = bid_depth/(bid_depth + ask_depth)
        
        q_ind = int(q*num_prob_steps)
    
    except StopIteration as e:
        raise e
    
    next_state = position, q_ind, price, entry_price
    
    return next_state, reward
                        

In [46]:
num_prob_steps

10

In [47]:
print(actions)
state = get_initial_state
print(step(START, 0))
print(step(START, 1))

{'buy': 2, 'sell': 0, 'hold': 1}
((1, 1, {'bid': 2238.1, 'ask': 2239.5}, 2239.5), 0)
((0, 5, {'bid': 2237.5, 'ask': 2239.5}, None), 0)


In [50]:
def choose_action(state, q_value,eps=epsilon):
    position, q, price, entry_price = state
    
    if np.random.binomial(1,eps) == 1:
        if position == positions['long']:
            action = np.random.choice([actions['hold'],actions['sell']])
        elif position == positions['short']:
            action = np.random.choice([actions['hold'],actions['buy']])
        else:
            action = np.random.choice([actions['hold'],actions['sell'],actions['buy']])
    
    else:
        if position == positions['long']:
            actions_ = [actions['hold'],actions['sell']]
        elif position == positions['short']:
            actions_ = [actions['hold'], actions['buy']]
        else:
            actions_ = [actions['hold'], actions['buy'], actions['sell']]
        
        values_ = q_value[state[0],state[1],actions_]
        
        action = np.random.choice([actions_[action_] 
                                   for action_, value_ in enumerate(values_)
                                   if value_ == np.max(values_)])
    return action

In [51]:
# Set a random state-action value function
q_value_example = np.random.random(Q_dims) 

# Show the initial state
state = get_initial_state(data_generator)
print(state)

# The action values for the initial state. 
# state[0] is the position; state[1] is the bid fill probability
print(q_value_example[state[0], state[1], :])

# With epsilon = 0, the selected action is always that with the highest Q-value
print(choose_action(state, q_value_example, eps=0))

(0, 5, {'bid': 2231.8, 'ask': 2239.5}, None)
[0.23148177 0.510653   0.57166512]
2


In [34]:
actions

[0, 1, 2]

In [52]:
max_iter

inf

In [ ]:
def sarsa(q_value, expected = False, step_size=Alpha, eps=epsilon):
    
    state = get_initial_state(data_generator)
    action = choose_action(state, q_value,eps)
    rewards = 0.0
    done = False
    iteration = 0
    
    while (iteration < max_iter) and not done:
        try:
            next_state, reward = step(state,action)
        except StopIteration:
            done=True
            continue